In [14]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20257, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 20257 (delta 9), reused 16 (delta 7), pack-reused 20237 (from 1)
Receiving objects: 100% (20257/20257), 277.35 MiB | 39.52 MiB/s, done.
Resolving deltas: 100% (741/741), done.
Updating files: 100% (137/137), done.


In [ ]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
print_system_info()

In [ ]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch torchvision
# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown
!uv pip install wandb
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

In [ ]:
# KAGGLE CELL #1: Download checkpoint

import os
import sys
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

In [ ]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

In [ ]:
import pandas as pd
import os

def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.

    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return

    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return

    all_characters = []
    # Ensure the column values are treated as strings before iterating over them
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")

# --- Example Usage (demonstration with a dummy file) ---
# As the original file 'Ds_300_ChuNom_TuTao.csv' was not found in the previous execution,
# let's create a dummy file to demonstrate the function's usage.
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")

# Create a dummy CSV file
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")

convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

# --- How to use with your actual file ---
# Uncomment the lines below and replace 'your_actual_file.csv' and 'your_output.txt'
# with the correct paths for your use case.
#
# original_csv_file = os.path.join(INPUT_PATH, "Ds_300_ChuNom_TuTao.csv") # Or the full path to your CSV
# original_output_txt = os.path.join(OUTPUT_PATH, "nom_tu_tao.txt") # Or your desired output path
# convert_csv_to_chars_txt(original_csv_file, original_output_txt)


In [ ]:
!ls -larth {OUTPUT_PATH}/ckpt

In [ ]:
%cd {OUTPUT_PATH}
from huggingface_hub import login
HF_TOKEN = load_secret("HF_TOKEN")
login(HF_TOKEN)
HF_USERNAME = "dzungpham"

# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================

# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN 
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN 
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_both" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_both" \
  --token HF_TOKEN 
# Validation: Seen Style, Unseen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_seen_style_unseen_char" \
  --token HF_TOKEN 
# Validation: Unseen Style, Seen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_style_seen_char" \
  --token HF_TOKEN 
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/test" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "test" \
  --token HF_TOKEN 
print("SUCCESSFULLY EXPORT HF DATASET TO LOCAL DIRECTORY")

In [ ]:
!uv pip install "huggingface-hub==0.25.2"

In [19]:
# already change sample_batch file to save all data in train_original
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --resume_from "my_dataset/train_original/results_checkpoint.json" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 76 \
    --end_line 100 \
    --batch_size 24 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

/kaggle/working
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-12-31 12:29:43.985251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767184184.006129    1483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767184184.012524    1483 cuda_blas.cc:1418] Unable to register cuBLAS factor

In [21]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --test_ratio 0.1 \
  --seed 42


FONTDIFFUSION VALIDATION SPLIT CREATOR
✓ Using source directory: my_dataset/train_original

CREATING DATA SPLITS

ANALYZING TRAINING DATA

✓ Found:
  Styles: 15
  Characters: 325
  Valid (char, style) pairs: 4875

✓ After validation:
  Characters with content images: 325
  Valid (char, style) pairs: 4875

CREATING VALIDATION SCENARIOS

📊 Split Statistics:
  Styles: 11 train + 3 val + 1 test
  Chars:  228 train + 65 val + 32 test

📋 Validation Scenarios:

  train:
    Description: Seen styles + Seen characters (training data)
    Styles: ['style13', 'style7', 'style2', 'style8', 'style11', 'style12', 'style4', 'style9', 'style14', 'style3', 'style10']
    Chars: ['char258', 'char230', 'char317', 'char83', 'char201']...

  val_seen_style_unseen_char:
    Description: Seen styles + Unseen characters
    Styles: ['style13', 'style7', 'style2', 'style8', 'style11', 'style12', 'style4', 'style9', 'style14', 'style3', 'style10']
    Chars: ['char249', 'char61', 'char47', 'char90', 'char229']

In [23]:
!uv pip install --upgrade pyarrow datasets

Using Python 3.11.13 environment at: /usr
Resolved 39 packages in 208ms                                        
Prepared 12 packages in 428ms                                            
Uninstalled 12 packages in 302ms
Installed 12 packages in 9.33s                              
 - anyio==4.11.0
 + anyio==4.12.0
 - certifi==2025.10.5
 + certifi==2025.11.12
 - click==8.3.0
 + click==8.3.1
 - datasets==4.4.1
 + datasets==4.4.2
 - filelock==3.20.0
 + filelock==3.20.1
 - huggingface-hub==0.25.2
 + huggingface-hub==1.2.3
 - numpy==1.26.4
 + numpy==2.4.0
 - pandas==2.2.3
 + pandas==2.3.3
 - pyarrow==19.0.1
 + pyarrow==22.0.0
 - typer-slim==0.20.0
 + typer-slim==0.21.0
 - tzdata==2025.2
 + tzdata==2025.3
 - urllib3==2.5.0
 + urllib3==2.6.2


In [24]:
# --- RAW DATA (Before Splitting) ---
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train_original" \
  --private \
  --token "{HF_TOKEN}"

# --- ORGANIZED SPLITS (After Splitting) ---

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train" \
  --private \
  --token "{HF_TOKEN}"

# Test Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/test" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "test" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Both
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_both" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_both" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Seen Style, Unseen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_seen_style_unseen_char" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Style, Seen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_style_seen_char" \
  --private \
  --token "{HF_TOKEN}"
print("SUCCESSFULLY UPLOAD LOCAL MY_DATASET TO HUGGINGFACE DATASETS SPACE")


FONTDIFFUSION DATASET CREATOR

Data dir: my_dataset/train_original
Repo: dzungpham/font-diffusion-generated-data
Push to Hub: True
✓ Validated directory structure
  Content images: my_dataset/train_original/ContentImage
  Target images: my_dataset/train_original/TargetImage

BUILDING DATASET

🖼️  Discovering images from disk...
  ✓ Found 325 content images
  ✓ Found 4875 target images

📋 Loading 4875 image pairs...
Loading images: 100%|███████████████████████████████████████| 4875/4875 [00:03<00:00, 1221.91pair/s]
✓ Loaded 4875 samples

PUSHING TO HUB
Repository: dzungpham/font-diffusion-generated-data
Split: train_original
Map: 100%|████████████████████████| 4875/4875 [00:00<00:00, 14103.23 examples/s]

Creating parquet from Arrow format: 100%|█████████| 1/1 [00:00<00:00,  6.02ba/s]
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

                              :   1%|▏   

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [27]:
# TRAINING PHASE 1
!uv pip install "huggingface-hub==0.25.2"
import wandb

# Load your Weights & Biases API key from a secure store
wandb_key = load_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

# Run the training script with the corrected flag syntax
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name=FontDiffuser_training_phase_1 \
    --data_root=my_dataset \
    --output_dir=outputs/FontDiffuser \
    --report_to=wandb \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=200 \
    --ckpt_interval=100 \
    --val_interval=200 \
    --gradient_accumulation_steps=1 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler=linear \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision=no

Using Python 3.11.13 environment at: /usr
Resolved 12 packages in 6ms                                          
Uninstalled 1 package in 4ms
Installed 1 package in 5ms25.2                              
 - huggingface-hub==0.36.0
 + huggingface-hub==0.25.2
Attempting to load secret 'WANDB_API_KEY' from 'kaggle' environment...
✅ Successfully loaded secret 'WANDB_API_KEY'.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()`

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")